# XGBOOST - GRIDSEARCH

In [1]:
import numpy as np
from scipy.stats import uniform as sp_rand
from sklearn import datasets
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
from datetime import datetime
from time import time

### GLOBAL VARIABLES

In [2]:
DATAPATH = 'data/train_test/'
SEED = 47
NITER = 100
CV = 3
SCORE = 'roc_auc'
handlingnull = False
NJOBS = 7
USEGPU = False

### LOAD DATASET

In [3]:
train_features = pd.read_pickle(DATAPATH+'X_train.pkl')

In [4]:
train_labels = pd.read_pickle(DATAPATH+'y_train.pkl')

In [5]:
train_features.shape

(152600, 1266)

In [ ]:
# drop var with same values
train_features = train_features.T.drop_duplicates().T

In [ ]:
train_labels.shape

In [6]:
### create a DMatrix and handling Null values
if handlingnull:
    #train_features[np.isnan(train_features)] = -9999
    xgtrain = xgb.DMatrix(train_features, train_labels, missing=-9999)
else:
    xgtrain = xgb.DMatrix(train_features, train_labels)

In [7]:
if handlingnull:
    train_features[np.isnan(train_features)] = -9999

### TRAIN MODEL

#### Set hyperparameters

In [10]:
# ======== General Parameters ======= #

# Select the type of model to run at each iteration. gbtree or gblinear.
booster = 'gbtree'


# Control the balance of positive and negative weights, useful for unbalanced classes. 
# A typical value to consider: sum(negative instances) / sum(positive instances)scale_pos_weight = 1
scale_pos_weight = int((len(train_labels) - np.sum(train_labels))/np.sum(train_labels))


# Learning Task Parameters

# This defines the loss function to be minimized. 
# - binary:logistic –logistic regression for binary classification, returns predicted probability (not class)
# - multi:softmax –multiclass classification using the softmax objective, returns predicted class (not probabilities)
#   you also need to set an additional num_class (number of classes) parameter defining the number of unique classes
# - multi:softprob –same as softmax, but returns predicted probability of each data point belonging to each class.
objective  = 'binary:logistic'


# The metric to be used for validation data.
# - rmse – root mean square error
# - mae – mean absolute error
# - logloss – negative log-likelihood
# - error – Binary classification error rate (0.5 threshold)
# - merror – Multiclass classification error rate
# - mlogloss – Multiclass logloss
# - auc: Area under the curve
eval_metric = 'auc'

#### Load hyperparameters

In [12]:
xgb_params = np.load('output/hyperparameters/rseach_xgboost_classifier_bestparams_d2019-11-08.npy', allow_pickle=True).tolist()

In [13]:
xgb_params['seed'] = SEED
xgb_params['booster'] = booster
xgb_params['objective'] = objective
xgb_params['eval_metric'] = eval_metric
xgb_params['num_threads'] = NJOBS
xgb_params['verbose'] = 0
xgb_params['scale_pos_weight'] = scale_pos_weight

In [15]:
if USEGPU:
    xgb_params['tree_method'] = 'gpu_hist'
    xgb_params['gpu_id'] = 0

In [16]:
xgb_params

{'subsample': 0.7,
 'reg_lambda': 0.7,
 'reg_alpha': 0.05,
 'min_child_weight': 9,
 'max_depth': 9,
 'learning_rate': 0.01,
 'gamma': 0.4,
 'colsample_bytree': 0.6,
 'seed': 47,
 'booster': 'gbtree',
 'objective': 'binary:logistic',
 'eval_metric': 'auc',
 'num_threads': 7,
 'verbose': 0,
 'scale_pos_weight': 9}

#### Find num boost

In [17]:
early_stopping_rounds = 50

In [18]:
cvresult = xgb.cv(xgb_params, xgtrain, num_boost_round = 1000, nfold = CV, metrics = eval_metric, early_stopping_rounds = early_stopping_rounds, seed = SEED)

[19:39:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 556 extra nodes, 0 pruned nodes, max_depth=9
[19:39:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 560 extra nodes, 0 pruned nodes, max_depth=9
[19:39:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 554 extra nodes, 0 pruned nodes, max_depth=9
[19:39:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 546 extra nodes, 0 pruned nodes, max_depth=9
[19:39:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 688 extra nodes, 2 pruned nodes, max_depth=9
[19:39:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 658 extra nodes, 2 pruned nodes, max_depth=9
[19:39:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 504 extra nodes, 0 pruned nodes, max_depth=9
[19:39:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 598 extra nodes, 0 pruned nodes, max_depth=9
[19:39:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 602 extra nodes, 0 pruned no

[19:40:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 532 extra nodes, 2 pruned nodes, max_depth=9
[19:40:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 614 extra nodes, 0 pruned nodes, max_depth=9
[19:40:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 648 extra nodes, 2 pruned nodes, max_depth=9
[19:40:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 494 extra nodes, 0 pruned nodes, max_depth=9
[19:40:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 650 extra nodes, 0 pruned nodes, max_depth=9
[19:40:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 536 extra nodes, 4 pruned nodes, max_depth=9
[19:40:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 560 extra nodes, 4 pruned nodes, max_depth=9
[19:40:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 554 extra nodes, 8 pruned nodes, max_depth=9
[19:40:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 520 extra nodes, 2 pruned no

[19:40:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 542 extra nodes, 0 pruned nodes, max_depth=9
[19:40:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 606 extra nodes, 0 pruned nodes, max_depth=9
[19:40:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 536 extra nodes, 0 pruned nodes, max_depth=9
[19:40:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 554 extra nodes, 4 pruned nodes, max_depth=9
[19:40:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 496 extra nodes, 4 pruned nodes, max_depth=9
[19:40:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 514 extra nodes, 0 pruned nodes, max_depth=9
[19:40:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 588 extra nodes, 0 pruned nodes, max_depth=9
[19:40:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 620 extra nodes, 2 pruned nodes, max_depth=9
[19:40:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 596 extra nodes, 2 pruned no

[19:40:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 556 extra nodes, 2 pruned nodes, max_depth=9
[19:40:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 594 extra nodes, 0 pruned nodes, max_depth=9
[19:40:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 504 extra nodes, 4 pruned nodes, max_depth=9
[19:40:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 464 extra nodes, 0 pruned nodes, max_depth=9
[19:40:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 648 extra nodes, 2 pruned nodes, max_depth=9
[19:40:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 536 extra nodes, 0 pruned nodes, max_depth=9
[19:40:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 350 extra nodes, 6 pruned nodes, max_depth=9
[19:40:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 498 extra nodes, 0 pruned nodes, max_depth=9
[19:40:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 574 extra nodes, 0 pruned no

[19:41:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 578 extra nodes, 2 pruned nodes, max_depth=9
[19:41:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 448 extra nodes, 4 pruned nodes, max_depth=9
[19:41:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 534 extra nodes, 0 pruned nodes, max_depth=9
[19:41:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 510 extra nodes, 4 pruned nodes, max_depth=9
[19:41:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 480 extra nodes, 4 pruned nodes, max_depth=9
[19:41:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 510 extra nodes, 0 pruned nodes, max_depth=9
[19:41:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 442 extra nodes, 0 pruned nodes, max_depth=9
[19:41:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 476 extra nodes, 0 pruned nodes, max_depth=9
[19:41:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 484 extra nodes, 0 pruned no

...

[19:48:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 154 extra nodes, 2 pruned nodes, max_depth=9
[19:48:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 220 extra nodes, 0 pruned nodes, max_depth=9
[19:48:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 212 extra nodes, 2 pruned nodes, max_depth=9
[19:48:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 194 extra nodes, 0 pruned nodes, max_depth=9
[19:48:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 2 pruned nodes, max_depth=9
[19:48:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=9
[19:48:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 294 extra nodes, 2 pruned nodes, max_depth=9
[19:48:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 148 extra nodes, 0 pruned nodes, max_depth=9
[19:48:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 290 extra nodes, 2 pruned nod

[19:49:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 178 extra nodes, 2 pruned nodes, max_depth=9
[19:49:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 216 extra nodes, 0 pruned nodes, max_depth=9
[19:49:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=9
[19:49:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 0 pruned nodes, max_depth=9
[19:49:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 2 pruned nodes, max_depth=9
[19:49:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=9
[19:49:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 136 extra nodes, 2 pruned nodes, max_depth=9
[19:49:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=9
[19:49:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 178 extra nodes, 2 pruned nodes

[19:49:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=9
[19:49:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 134 extra nodes, 0 pruned nodes, max_depth=9
[19:49:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 260 extra nodes, 4 pruned nodes, max_depth=9
[19:49:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 162 extra nodes, 2 pruned nodes, max_depth=9
[19:49:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 130 extra nodes, 0 pruned nodes, max_depth=9
[19:49:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 138 extra nodes, 2 pruned nodes, max_depth=9
[19:49:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned nodes, max_depth=9
[19:49:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 190 extra nodes, 2 pruned nodes, max_depth=9
[19:49:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nod

[19:49:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 206 extra nodes, 2 pruned nodes, max_depth=9
[19:49:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 278 extra nodes, 0 pruned nodes, max_depth=9
[19:49:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 2 pruned nodes, max_depth=9
[19:49:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 0 pruned nodes, max_depth=9
[19:49:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 260 extra nodes, 0 pruned nodes, max_depth=9
[19:49:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 204 extra nodes, 6 pruned nodes, max_depth=9
[19:49:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=9
[19:49:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 248 extra nodes, 2 pruned nodes, max_depth=9
[19:49:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned node

[19:50:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 216 extra nodes, 0 pruned nodes, max_depth=9
[19:50:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 170 extra nodes, 0 pruned nodes, max_depth=9
[19:50:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=9
[19:50:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 226 extra nodes, 0 pruned nodes, max_depth=9
[19:50:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 0 pruned nodes, max_depth=9
[19:50:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 246 extra nodes, 6 pruned nodes, max_depth=9
[19:50:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=9
[19:50:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=9
[19:50:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 262 extra nodes, 0 pruned node

In [19]:
n_estimators = cvresult.shape[0]

In [20]:
print("Best number of boosters: ", n_estimators)

Best number of boosters:  622


#### Set hyperparameters

In [22]:
# ======== Booster Parameters ======== # 

# Analogous to learning rate in GBM. 
# Typical final values to be used: 0.01-0.2
eta = [0.01] 

# Defines the minimum sum of weights of all observations required in a child.
min_child_weight = [xgb_params['min_child_weight']]

# The maximum depth of a tree
max_depth = [i for i in range(int(xgb_params['max_depth']) - 1, int(xgb_params['max_depth']) + 2, 1)]

# A node is split only when the resulting split gives a positive reduction in the loss function. 
# Gamma specifies the minimum loss reduction required to make a split.
gamma = [xgb_params['gamma']]

# Denotes the fraction of observations to be randomly samples for each tree.
subsample = [i/100 for i in range(int(xgb_params['subsample']*100) - 4, int(xgb_params['subsample']*100) + 6, 3)]

# Denotes the fraction of columns to be randomly samples for each tree.
colsample_bytree = [i/100 for i in range(int(xgb_params['colsample_bytree']*100) - 4, int(xgb_params['colsample_bytree']*100) + 6, 3)]

# L2 regularization term on weights (analogous to Ridge regression)
reg_lambda =  [i/100 for i in range(int(xgb_params['reg_lambda']*100) - 4, int(xgb_params['reg_lambda']*100) + 4, 4)]

# L1 regularization term on weight (analogous to Lasso regression)
reg_alpha = [xgb_params['reg_alpha']]

# Control the balance of positive and negative weights, useful for unbalanced classes. 
# A typical value to consider: sum(negative instances) / sum(positive instances)scale_pos_weight = 1
scale_pos_weight = int((len(train_labels) - np.sum(train_labels))/np.sum(train_labels))

[xgboost params](https://xgboost.readthedocs.io/en/latest/python/python_api.html)

In [23]:
# Create the random grid
param_grid = {
    'learning_rate' : eta,
    'min_child_weight' : min_child_weight,
    'max_depth' : max_depth,
    'gamma': gamma,
    'subsample': subsample,
    'colsample_bytree' : colsample_bytree,
    'reg_lambda' : reg_lambda,
    'reg_alpha' : reg_alpha,
}

In [24]:
param_grid

{'learning_rate': [0.01],
 'min_child_weight': [9],
 'max_depth': [8, 9, 10],
 'gamma': [0.4],
 'subsample': [0.66, 0.69, 0.72, 0.75],
 'colsample_bytree': [0.56, 0.59, 0.62, 0.65],
 'reg_lambda': [0.66, 0.7],
 'reg_alpha': [0.05]}

In [25]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
model = XGBClassifier(n_estimators=n_estimators, 
                      scale_pos_weight=scale_pos_weight, 
                      #num_class=3, 
                      objective=objective)

In [26]:
if USEGPU:
    model.set_params(gpu_id = 0)
    model.set_params(tree_method='gpu_hist')

In [29]:
# Random search of parameters, using CV fold cross validation, 
# search across NITER different combinations, and use all available cores
xgboost_gsearch = GridSearchCV(estimator = model, param_grid = param_grid, scoring=SCORE, cv = CV, verbose=1,  n_jobs = NJOBS)# Fit the random search model


#### Training

In [ ]:
start = time()
xgboost_gsearch.fit(train_features, train_labels)
print("GridSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), NITER))

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed: 136.9min


#### Saving results

In [31]:
cv_results = pd.DataFrame(xgboost_gsearch.cv_results_)

/opt/anaconda3/envs/xgboostenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/anaconda3/envs/xgboostenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/anaconda3/envs/xgboostenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/an

In [32]:
cv_results.to_csv('output/results/gsearch_xgboost_classifier_d' + str(datetime.now().date()) + '.csv',sep=';',index=False)

#### Best estimator

In [33]:
xgboost_gsearch.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.59, gamma=0.4, learning_rate=0.01,
       max_delta_step=0, max_depth=8, min_child_weight=9, missing=None,
       n_estimators=622, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.05,
       reg_lambda=0.66, scale_pos_weight=9, seed=None, silent=True,
       subsample=0.66)

#### Best parameter

In [36]:
best_params = xgboost_gsearch.best_params_

In [37]:
best_params['n_estimators'] = n_estimators

In [38]:
best_params

{'colsample_bytree': 0.59,
 'gamma': 0.4,
 'learning_rate': 0.01,
 'max_depth': 8,
 'min_child_weight': 9,
 'reg_alpha': 0.05,
 'reg_lambda': 0.66,
 'subsample': 0.66,
 'n_estimators': 622}

#### Best Score

In [35]:
print(SCORE,' : ', xgboost_gsearch.best_score_)

roc_auc  :  0.7596728366676243


#### Saving best model

In [39]:
np.save('output/hyperparameters/gseach_xgboost_classifier_bestparams_d' + str(datetime.now().date()) + '.npy', best_params)

In [41]:
np.save('output/results/gseach_xgboost_classifier_best_estimator_d' + str(datetime.now().date()) + '.npy', xgboost_gsearch.best_estimator_)